In [ ]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

In [ ]:
import pandas as pd
data = pd.read_csv("drive/MyDrive/Сертификат/news-dataset/assets/test.csv",header=None)

In [ ]:
values  = data.values

In [ ]:
for i in range(len(values)):
    values[i][0]=i

In [ ]:
values

In [ ]:
import random
for i in range(10):
  x = random.randint(0,7599)
  print(values[x][2])
  print()

In [ ]:
# values

In [ ]:
all_embs =  []
from tqdm import tqdm
for el in tqdm(values):
    embs = model.encode([el[2]], normalize_embeddings=True)
    all_embs.append((el[0],embs))

In [ ]:
all_embs[0][1][0].tolist()

In [ ]:
!pip install chromadb

In [ ]:
!pip install --upgrade --force-reinstall fastapi==0.103.2
#!pip install typing-extensions --upgrade

In [ ]:
import chromadb
client = chromadb.Client()

#collection = client.create_collection("db0")

collection_l2 = client.create_collection("l2_db", {"hnsw:space": "l2"})
collection_cos = client.create_collection('cos', {"hnsw:space": "cosine"})
collection_Ip = client.create_collection('Ip_db', {"hnsw:space": "ip"})


In [ ]:
abc = [el[1][0].tolist() for el in all_embs]

In [ ]:
collection_l2.add(
    embeddings=abc,
    metadatas=[{"headline":el} for el in values[:,1].tolist()],
    ids=[str(el[0]) for el in all_embs]
)

collection_cos.add(
    embeddings=abc,
    metadatas=[{"headline":el} for el in values[:,1].tolist()],
    ids=[str(el[0]) for el in all_embs]
)

collection_Ip.add(
    embeddings=abc,
    metadatas=[{"headline":el} for el in values[:,1].tolist()],
    ids=[str(el[0]) for el in all_embs]
)

In [ ]:
import random
random_elements = [[el[0],el[2]] for el in random.choices(values,k=10)]

In [ ]:
random_elements

In [ ]:
questions_array = [
    (6983, "What could be the consequences for Rwanda if it does not comply with the UN Security Council’s demand for the immediate withdrawal of its troops from the Democratic Republic of Congo?"),
    (5994, "What event caused stocks to rise on Wednesday, and what was the reaction of investors to the indicator of rising inflation and the merger of Kmart Holding Corp?"),
    (3180, "What action does the Russian government expect its parliament to take this month regarding the Kyoto Protocol, and what would be the global impact of this action?"),
    (2041, "What event 8,200 years ago caused a significant cooling of the North Atlantic region, and what are scientists currently trying to predict?"),
    (6888, "Who formed a productive practice pair in 1998 for the Patriots and Cleveland, and what were their individual achievements in that year?"),
    (3094, "Who was the head coach of the Philadelphia Eagles from 1961-63 and the offensive line coach on the 1960 championship team, and what recent event occurred related to him?"),
    (7239, "What event has caused the maiden flight of Boeing’s new heavy-lift Delta 4 rocket to be delayed by a week?"),
    (1919, "What three events delivered staggering blows to the Gulf Coast, Florida, and Caribbean island nations all within a span of five weeks?"),
    (5597, "What event led to a 16 percent rise in profit for British satellite broadcaster BSkyB in the first quarter, and what was the context of this growth?"),
    (1340, "What is the impact of success in big-time sports on a college’s alumni donations and the academic quality of its applicants, according to a study directed by the Knight Commission on Intercollegiate Athletics?"),
    (1345, "What is my name?")
]

In [ ]:
instruction = "Represent this sentence for searching relevant passages:"
search_results = []
for q in questions_array:
    query = instruction + " " + q[1]
    query = model.encode(query)#, normalize_embeddings=True)
    results = collection_l2.query(query_embeddings=[query.tolist()],n_results=7600) #l2 + normalize = cos distance
    search_results.append(results["ids"][0].index(str(q[0])))

In [ ]:
print(search_results)
from statistics import mean
mean(search_results)

In [ ]:
instruction = "Represent this sentence for searching relevant passages:"
search_results = []
for q in questions_array:
    query = instruction + " " + q[1]
    query = model.encode(query)#, normalize_embeddings=True)
    results = collection_cos.query(query_embeddings=[query.tolist()],n_results=7600) #l2 + normalize = cos distance
    search_results.append(results["ids"][0].index(str(q[0])))

In [ ]:
print(search_results)
from statistics import mean
mean(search_results)

In [ ]:
instruction = "Represent this sentence for searching relevant passages:"
search_results = []
for q in questions_array:
    query = instruction + " " + q[1]
    query = model.encode(query)#, normalize_embeddings=True)
    results = collection_Ip.query(query_embeddings=[query.tolist()],n_results=7600) #l2 + normalize = cos distance
    search_results.append(results["ids"][0].index(str(q[0])))

In [ ]:
print(search_results)
from statistics import mean
mean(search_results)

In [ ]:
#results["ids"][0].index(str(4498))